In [82]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import numpy as np
import scipy
from collections import defaultdict

In [2]:
with open('Airbnb_Texas_Rentals_new.csv','r',encoding="utf8") as csv_file:
    csv_reader=csv.reader(csv_file)
    
    with open('Airbnb_Texas_Rentals.tsv','w',encoding="utf8") as new_file: 
        csv_writer=csv.writer(new_file,delimiter='\t')
    
        for line in csv_reader:
               csv_writer.writerow(line[1:]) 

with open('Airbnb_Texas_Rentals.tsv','r',encoding="utf8") as all_files:
    for i, row in enumerate(all_files):
        if i > 0 and i < 40:
            if row != '\n':
                with open('doc\doc_' + str(int((i-(i/2)))) + '.tsv','w',encoding="utf8") as new_files:
                    new_files.write(row)      

In [3]:
def save_dict_to_file(dic, file):
    f = open('{}.txt'.format(file), 'w',encoding="utf8")
    f.write(str(dic))
    f.close()

In [ ]:
docpaths = [f for f in listdir("doc") if isfile(join("doc", f))]

documentlist = {}
vocabulary = {}
inverted_index = {}
indexOfWord = 0
for doc in docpaths:
    with open('doc\{}'.format(doc), 'r',encoding="utf8") as file:
        lines = file.readlines()
        if len(lines) > 0 :
            document = lines[0]
            cols = document.replace('\\n',' ').split('\t')[:-1]
            document = '\t'.join(cols)
            tokens = nltk.word_tokenize(document)
            filtered_words = [nltk.stem.PorterStemmer().stem(word) for word in tokens if word not in nltk.corpus.stopwords.words('english') and word not in ".,'()"]
            documentlist[doc] = filtered_words
            for w in filtered_words:
                if w not in vocabulary:
                    indexOfWord += 1
                    vocabulary[w] = indexOfWord
                if vocabulary[w] not in inverted_index:
                    temp = []
                    temp.append(doc)
                    inverted_index[vocabulary[w]] = temp
                elif doc not in inverted_index[vocabulary[w]]:
                    inverted_index[vocabulary[w]].append(doc)


save_dict_to_file(inverted_index,"inverted_index")
save_dict_to_file(vocabulary,"vocabulary")
save_dict_to_file(documentlist,"documentlist")

In [5]:
query = input()
tokens = nltk.word_tokenize(query)
query_tokens = [nltk.stem.PorterStemmer().stem(token) for token in tokens if token not in stopwords.words('english') if token not in ".,'()"]

vocabulary = open('vocabulary.txt', 'r', encoding = 'utf-8') 
L1 = eval(vocabulary.read())
inverted_index = open('inverted_index.txt', 'r', encoding = 'utf-8')  
L2 = eval(inverted_index.read())
        
term_ids = [L1[token] for token in query_tokens if token in L1]
search_results = [L2[idx] for idx in term_ids if idx in L2]  

new_list = [set(list_) for list_ in search_results]
intersect = set.intersection(*new_list)

col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url"]
df = pd.DataFrame(columns = col_names)#index = False?

for j, doc in enumerate(intersect):
    with open(r'doc\{}'.format(doc), 'r',encoding="utf-8") as file:
        
        csv_reader = csv.reader(file, delimiter = '\t')
        for line in csv_reader:
            line = line[1:]
            for i, field in enumerate(line):
                df.loc[j, col_names[i]] = field 
        
pd.set_option('max_colwidth',500)
display(df[['title', 'description', 'city', 'url']])

room bedroom


,title,description,city,url
0,Friendly Private Room in َQuiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX
1,Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. \n\nAmazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. \n\nFeaturing an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more\n\nEuropean inspired kitchen and “top of the line” decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX
2,2 Private rooms/bathroom 10min from IAH airport,Welcome to stay in private room with queen bed and detached private bathroom on the second floor. Another private bedroom with sofa bed is available for additional guests. 10$ for an additional guest.\n10min from IAH airport\nAirport pick-up/drop off is available for $10/trip.,Humble,https://www.airbnb.com/rooms/18520444?location=Cleveland%2C%20TX
3,Private Room Close to Campus,Private bedroom in a cute little home situated in the coveted Garden Acres neighborhood in Bryan. The bedroom has its own private access and its own private bathroom.,Bryan,https://www.airbnb.com/rooms/11839729?location=College%20Station%2C%20TX


In [7]:
#how may times the word is repeated within the i-th document divided by its length
def tf(word, document):
    return document.count(word) / len(document)

def word_containing(word, documentlist):
    return sum(1 for document in documentlist if word in document)

#IDF(word) = 1 + loge(Total Number Of Documents / Number Of Documents with term 'word' in it)
def idf(word, documentlist):
    return math.log(len(documentlist) / (1 + word_containing(word, documentlist)))

def tfidf(word, document, documentlist):
    return tf(word, document) * idf(word, documentlist)

In [69]:
new_inverted_index = {}
with open('documentlist.txt', 'r', encoding="utf-8") as file:
    data = eval(file.read())
    for key,doc in documentlist.items():
        for w in doc:
            score = tfidf(w,doc,documentlist)
            w_index = (key,score)
            if vocabulary[w] not in new_inverted_index:
                temp = []
                temp.append(w_index)
                new_inverted_index[vocabulary[w]] = temp
            elif doc not in new_inverted_index[vocabulary[w]]:
                new_inverted_index[vocabulary[w]].append(w_index)
#print(new_inverted_index)
for key in new_inverted_index:
    new_inverted_index[key] = list(set(new_inverted_index[key]))

save_dict_to_file(new_inverted_index,"inverted_index_tfidf")

In [47]:

query = input()
tokens = nltk.word_tokenize(query)
query_tokens = [nltk.stem.PorterStemmer().stem(token) for token in tokens if token not in stopwords.words('english') if token not in ".,'()"]

tfidf_query_array = []
for w in query_tokens:
    # tf/idf score for the query: the tf will just be how many times the word is contained in the query,
    # the idf will be just 1 + the logarithm of the number of documents(1) divided by 1 (because the term is repeated only in 1 document that's the query)
    # so the idf will be 1 + 0 = 1
    score = tf(w, query_tokens)
    tfidf_query_array.append(score)
print(tfidf_query_array)

room house
[0.5, 0.5]


In [121]:
inverted_index_tfidf = open("inverted_index_tfidf.txt", 'r', encoding = 'utf-8')  
L2 = eval(inverted_index_tfidf.read())


term_ids = [L1[token] for token in query_tokens if token in L1]
print(term_ids)
search_results = [L2[idx] for idx in term_ids if idx in L2]
print(search_results)
new_list = [set(list_) for list_ in search_results]
intersect = set.intersection(*new_list)
print(intersect)

cos_arrays = {}
for list_ in search_results:
    for tuple_ in list_:
        if tuple_[0] not in cos_arrays:
            temp = []
            temp.append(tuple_[1])
            cos_arrays[tuple_[0]] = temp
        else:
            cos_arrays[tuple_[0]].append(tuple_[1])
print(cos_arrays)

final = [cos_arrays[key] for key in cos_arrays if len(cos_arrays[key]) == len(term_ids)]
print(final)


[11, 196]
[[('doc_12.tsv', 0.035052844990076666), ('doc_2.tsv', 0.037749217681621025), ('doc_4.tsv', 0.09814796597221467), ('doc_1.tsv', 0.0640095430253574), ('doc_8.tsv', 0.07361097447916101), ('doc_3.tsv', 0.09814796597221467), ('doc_14.tsv', 0.044612711805552124)], [('doc_2.tsv', 0.037749217681621025), ('doc_16.tsv', 0.1280190860507148), ('doc_8.tsv', 0.07361097447916101), ('doc_3.tsv', 0.29444389791664405), ('doc_14.tsv', 0.044612711805552124)]]
{('doc_2.tsv', 0.037749217681621025), ('doc_8.tsv', 0.07361097447916101), ('doc_14.tsv', 0.044612711805552124)}
{'doc_12.tsv': [0.035052844990076666], 'doc_2.tsv': [0.037749217681621025, 0.037749217681621025], 'doc_4.tsv': [0.09814796597221467], 'doc_1.tsv': [0.0640095430253574], 'doc_8.tsv': [0.07361097447916101, 0.07361097447916101], 'doc_3.tsv': [0.09814796597221467, 0.29444389791664405], 'doc_14.tsv': [0.044612711805552124, 0.044612711805552124], 'doc_16.tsv': [0.1280190860507148]}
[[0.037749217681621025, 0.037749217681621025], [0.07361

In [122]:
for array in final:
    sim = 1 - (scipy.spatial.distance.cosine(np.array(tfidf_query_array) , np.array(array)))
    print(sim)

1.0
1.0
0.8944271909999157
1.0
